In [1]:
import pandas as pd
import numpy as np

In [1]:
df = pd.read_excel(r'C:\Users\dfrigerio\Desktop\model\indica_au_pat.xlsx')

cols = ['cod_cia','nombre','periodo', 'ind_04', 'ind_20', 'ind_21', 'ind_19','ind_32','ind_42','ind_22']
ind_a_incr = ['ind_04', 'ind_20', 'ind_21', 'ind_19','ind_32','ind_42']

#Quedarnos unicamente con los indicadores buscados
ind = df[cols].copy()

#Reemplazar los Nan por 0
ind.fillna(value = 0, inplace = True)

#Reemplazar el campo periodo
ind['periodo'] = ind['periodo'].apply(lambda x: int(x.replace('-','0')))

ind.sort_values(by = ['cod_cia','periodo'], inplace = True)
ind.reset_index(drop = True, inplace = True)

#Trabajar con los incrementos
grupo_inc = ind.groupby('cod_cia')

for p in [1,2,3,4]:
    sufijo = str(p)
    for i in ind_a_incr:
        nombre = i+'_'+sufijo
        ind[nombre] = grupo_inc[i].pct_change(periods = int(sufijo))
        del nombre
    del sufijo

#Ver deficit
ind_inv = ind[cols].sort_values(by = ['cod_cia','periodo'], ascending = [True,False]).copy()
grupo_inv = ind_inv.groupby('cod_cia')['ind_22']

for p in [1,2,3,4]:
    nombre = 'solv_'+str(p)
    ind_inv[nombre] = grupo_inv.shift(int(p))
    del nombre
    
#Reordenar nuevamente las filas
ind_inv.sort_values(by = ['cod_cia','periodo'], inplace = True)
ind_inv.reset_index(drop = True, inplace = True)

#Generar la columna de déficit o superavit de CM para el año siguiente de observacion. 1 Deficit 0 Superavit

ind_inv['sup_def'] = np.where((ind_inv['solv_1']<0)|(ind_inv['solv_2']<0)|(ind_inv['solv_3']<0)|(ind_inv['solv_4']<0)
                              ,1,0)

#Juntar el set original con la columna de sup_deficit
conjunto = ind.copy()
conjunto['sup_def'] = ind_inv['sup_def']

#Generamos la base final únicamente cuando el período de observación no tiene déficit y en el año siguiente SI
base = conjunto.loc[~((conjunto['ind_22']<0) & (conjunto['sup_def']==1)) & 
           (conjunto['periodo']<201803) & (conjunto['periodo']>201301)].copy()

base.reset_index(drop = True, inplace = True)
    
maximo_reemplazo = {i:base[base[i]!= np.inf][i].max() for i in base.columns[10:]}
minimo_reemplazo = {i:base[base[i]!= -np.inf][i].min() for i in base.columns[10:]}

for i in base.columns[10:]:
    base[i] = base[i].replace(np.inf,maximo_reemplazo[i])
    base[i] = base[i].replace(-np.inf,minimo_reemplazo[i])

NameError: name 'pd' is not defined

In [3]:
print(ind.shape)
print(ind_inv.shape)
print(base.shape)

(2393, 34)
(2393, 15)
(1662, 35)


In [4]:
base.sup_def.value_counts(normalize = True)

0    0.93562
1    0.06438
Name: sup_def, dtype: float64

In [5]:
base.sort_values(by = ['periodo'], inplace = True)
base.reset_index(drop = True, inplace = True)

In [6]:
base.periodo.unique()

array([201302, 201303, 201304, 201401, 201402, 201403, 201404, 201501,
       201502, 201503, 201504, 201601, 201602, 201603, 201604, 201701,
       201702, 201703, 201704, 201801, 201802], dtype=int64)

## Predicciones rápidas

In [7]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [8]:
testeo = base.loc[base['periodo']==201802, base.columns[3:]].copy()
testeo.fillna(value=0,inplace = True)
X_test = testeo.loc[:, testeo.columns!='sup_def'].copy()
y_test = testeo['sup_def']

entrenamiento = base.loc[base['periodo']!=201802, base.columns[3:]].copy()
entrenamiento.fillna(value=0,inplace = True)
X_train = entrenamiento.loc[:, entrenamiento.columns!='sup_def'].copy()
y_train = entrenamiento['sup_def']

print('Entrenamiento X: ',X_train.shape)
print('Entrenamiento y: ',y_train.shape)
print('Testeo X: ',X_test.shape)
print('Testeo y: ',y_test.shape)

Entrenamiento X:  (1581, 31)
Entrenamiento y:  (1581,)
Testeo X:  (81, 31)
Testeo y:  (81,)


In [9]:
scalar = StandardScaler()
X_tr_sca = scalar.fit_transform(X_train)
X_tst_sca = scalar.fit_transform(X_test)

In [28]:
clf = SGDClassifier(loss="log", penalty="l2", max_iter=1000)

In [29]:
clf.fit(X_tr_sca, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1000,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [30]:
accuracy_score(y_true = y_test, y_pred = clf.predict_proba(X_tst_sca)[:,1].round(), normalize = False)

65

In [31]:
clf.predict_proba(X_tst_sca)[:,1].round()

array([1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [20]:
y_test.shape

(81,)